In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

2024-10-07 14:37:18.918367: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-07 14:37:18.918715: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-07 14:37:18.920587: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-07 14:37:18.926226: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-07 14:37:18.935567: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
def read_csv_to_df(path):
    return pd.read_csv(path)

# should try filtered set also in case its better (but I dont know if the data was filtered prior or not)
h_data_path = "healthy.csv"
p_data_path = "pathology.csv"

h_df = read_csv_to_df(h_data_path)
p_df = read_csv_to_df(p_data_path)

h_data = []
p_data = []

for spectrogram_id, group in h_df.groupby('spectrogram_id'):
    # Extract frequency, time, and power values
    power = []
    for f, g in group.groupby("frequency"):
        p = g['power']
        p_num = p.to_numpy()
        power.append(p_num)
    p_ = np.array(power)
    p_shaped = p_.reshape(p_.shape[0], p_.shape[1], 1)  # Shape will be (129, 41, 1)
    h_data.append(p_shaped)

for spectrogram_id, group in p_df.groupby('spectrogram_id'):
    # Extract frequency, time, and power values
    power = []
    for f, g in group.groupby("frequency"):
        p = g['power']
        p_num = p.to_numpy()
        power.append(p_num)
    p_ = np.array(power)
    p_shaped = p_.reshape(p_.shape[0], p_.shape[1], 1)  # Shape will be (129, 41, 1)
    p_data.append(p_shaped)

In [11]:
# Stack the arrays for class 1 and class 0
X_class_1 = np.stack(h_data)  # Shape will be (n_samples_1, 129, 41, 1)
X_class_0 = np.stack(p_data)  # Shape will be (n_samples_0, 129, 41, 1)

# Print shapes to verify
print("Shape of X_class_1:", X_class_1.shape)  # Should show (n_samples_1, 129, 41, 1)
print("Shape of X_class_0:", X_class_0.shape)  # Should show (n_samples_0, 129, 41, 1)

# Concatenate the stacked arrays
X = np.concatenate((X_class_0, X_class_1), axis=0)  # Shape will be (total_samples, 129, 41, 1)
print("Shape of X:", X.shape) 

y_class_0 = np.zeros(X_class_0.shape[0])  # Labels for class 0
y_class_1 = np.ones(X_class_1.shape[0])   # Labels for class 1
y = np.concatenate((y_class_0, y_class_1))  # Combine labels

Shape of X_class_1: (51, 129, 41, 1)
Shape of X_class_0: (38, 129, 41, 1)
Shape of X: (89, 129, 41, 1)


In [12]:
# Shuffle the dataset - this is done by the .fit method potentially?
indices = np.arange(X.shape[0])  # Create an array of indices
np.random.shuffle(indices)  # Shuffle indices
X = X[indices]  # Shuffle the data
y = y[indices]  # Shuffle the labels

y = np.array(y, dtype=np.int32)  # Convert labels to int32
X = np.array(X)  # it is already an np.array but making absolutely sure

# verify shapes of data and types
print("Shape of X:", X.shape)  # Should be (total_samples, 129, 41, 1)
print("Shape of y:", y.shape)  # Should be (total_samples,)
print("Type of X:", type(X))  # Should be <class 'numpy.ndarray'>
print("Type of y:", type(y))  # Should be <class 'numpy.ndarray'>
print("y dtype:", y.dtype)  # Should show int32

Shape of X: (89, 129, 41, 1)
Shape of y: (89,)
Type of X: <class 'numpy.ndarray'>
Type of y: <class 'numpy.ndarray'>
y dtype: int32


#### Define the CNN Model

In [13]:
# try sigmoid and softmax

#### last results (restart kernal first) ##################
# validation data of 0.1, batch_size 32
# 107ms/step - accuracy: 0.9607 - loss: 0.0576  
# Loss: 0.11523419618606567, Accuracy: 0.9213483333587646
###########################################################

#### last results (restart kernal first) ##################
# validation data of 0.3, batch_size 32
# 105ms/step - accuracy: 0.5795 - loss: 0.6805
# Loss: 0.6825977563858032, Accuracy: 0.5730336904525757
###########################################################

def generate_model(activation):
    a = (0,"")
    if(activation == "softmax"):
        a = (2, activation)
    elif(activation == "sigmoid"):
        a=(1, activation)
    print(a[1])
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', ), # no input_shape=(129,41,1)
        MaxPooling2D(pool_size=2, strides=1),

        Conv2D(64, (3, 3), activation='relu', ), # no input_shape=(129,41,1)
        MaxPooling2D(pool_size=2, strides=1),

        Flatten(),
        Dense(1024, activation='relu'),
        Dense(a[0], activation=a[1])
    
    ])

    return model

activation = "softmax"
model = generate_model(activation)
print(model)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

softmax
<Sequential name=sequential_2, built=False>


#### Train the model

In [ ]:
# Train the model
if(activation == "softmax"):
    y = to_categorical(y, num_classes=2)  # Convert to one-hot encoding

history = model.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.6153 - loss: 502.5535 - val_accuracy: 0.2778 - val_loss: 233.5586
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.4957 - loss: 152.5383 - val_accuracy: 0.7222 - val_loss: 41.4104
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5574 - loss: 30.8514 - val_accuracy: 0.2778 - val_loss: 24.1027
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.4785 - loss: 15.6320 - val_accuracy: 0.7222 - val_loss: 1.1886
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5254 - loss: 1.6714 - val_accuracy: 0.7222 - val_loss: 0.6346
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5364 - loss: 0.6915 - val_accuracy: 0.7222 - val_loss: 0.7536
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.4441 - loss: 1.1418 - val_accuracy: 0.6667 - val_loss: 0.6925
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.5567 - loss: 0.6929 - val_accuracy: 0.7222 - val_loss: 0.6917
Epoch 

#### Evaluate Model

In [8]:
loss, accuracy = model.evaluate(X, y)
print(f"Loss: {loss}, Accuracy: {accuracy}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.5756 - loss: 0.6818
Loss: 0.6824779510498047, Accuracy: 0.5730336904525757
